# Birch-Murnaghan Equation of State Fitting

I just re-write the code that have been made before by my supervisor in Matlab into Python.  You can download the folder containing `inpVE.txt`, `BMEOS.py`, and a note in [my_Gdrive](https://drive.google.com/drive/folders/1trbXpaZkUBEubB5ekmbHlZsLWab6Y4Q_?usp=sharing). By using this code, you can fit the non linear curve of your volume vs energy data using Birch-Murnaghan Equation of State, and it will return `B0, dB0, V0, E0.txt` and a `poly-eos curve` as the output. The explanation about the code is in the below.

In [ ]:
from pylab import *
import scipy.optimize 

#Load your volume and energy data from inpVE.txt
V=np.loadtxt("inpVE.txt")[:, 0]
E=np.loadtxt("inpVE.txt")[:, 1]

# second order polynomial fitting using pylab and get the coefficients (a,b c)
p=polyfit(V,E,2)
a=p[0]
b=p[1]
c=p[2]

E0 = a*V0**2+b*V0+c
V0 = -b/(2*a) # dE/dV = 0 (V0 is the minimum energy volume) -> 2*a*V0 + b = 0 -> V0=-b/2a
B0 = 2*a*V0 # B0 = V0*(d**2E/dV**2)
dB0 = 4

x0 = [B0, dB0, V0, E0] #initial guess obtained from the second order polynomial fitting

#define the Birch-Murnaghan Equation of States function
def func_birchmur(VOL, x):
    
    vv= (x[2]/VOL)**(1.0/3.0)
    E =  x[3] + 9.0/16.0*x[2]*x[0] *(vv**2-1.0)**2 * (6.0 + x[1]*(vv**2-1.0) - 4.0*vv**2)
    
    return E

sigma= lambda x: sum((E - func_birchmur(V,x))**2) #the chi squared function
x_final = scipy.optimize.fmin(sigma, x0) #minimized the chi squared function and return the minimum values of constants [B0, dB0, V0, E0] 

#plotting the polymonial and BMEOS curve and save the curve as poly-eos.png
x_graph = np.linspace(min(V),max(V),1000)
plot(x_graph, a*x_graph**2 + b*x_graph + c,'--',label='polinomial fitting')
plot(x_graph, func_birchmur(x_graph, x_final), label='EOS fitting')
plot(V, E, 'bo')
legend(loc='best')
savefig('poly-eos.png')
show()

#writing the "B0_dB0_V0_E0" text file containing the minimum value of constants
f= open("B0_dB0_V0_E0.txt","w+")
f.write('%.10f  %.10f  %.10f  %.10f \n' %(x_final[0],x_final[1],x_final[2],x_final[3]))
f.close()



